In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import pickle
import random

In [2]:
# game data outputted from W1_DataCleaning.ipynb, with stats, goes back to 2010
game_data = pd.read_csv('mydata/womens/matchups.csv')
game_data.head()

,Season,MooreRating_x,MooreRating_y,Seed_x,3ptRate_x,Ast%_x,FT%_x,OppFT%_x,Opp3ptRate_x,OppAst%_x,...,xOffyOffFTRateDiff,AbsxOffyDefAstDiff,AbsyOffxDefAstDiff,xOffyDefAstAvg,yOffxDefAstAvg,xOffyOffAstDiff,TotalPossVarSum,GameScoreVarSum,AvgTotalPoss,MooreNaiveUpsetProbability
0,2010.0,91.81,78.17,4,0.153633,0.562874,0.726562,0.687204,0.318363,0.449704,...,0.103332,0.046362,0.116533,0.539693,0.507970,0.031723,323.329209,249.795260,142.390152,0.111138
1,2010.0,91.81,82.29,4,0.153633,0.562874,0.726562,0.687204,0.318363,0.449704,...,0.137651,0.109423,0.216030,0.617586,0.557719,0.059867,498.758776,240.489802,139.451000,0.192651
2,2010.0,95.45,91.81,1,0.230377,0.550165,0.678383,0.679487,0.302521,0.512938,...,-0.085583,0.100461,0.049937,0.499935,0.537906,-0.037971,356.530636,259.872359,139.373125,0.374739
3,2010.0,92.39,91.81,2,0.226961,0.530850,0.662016,0.665517,0.309621,0.603648,...,-0.118697,0.081146,0.040774,0.490277,0.583261,-0.092984,359.897075,306.882580,141.712500,0.481327
4,2010.0,114.80,91.81,1,0.276465,0.623000,0.723077,0.630556,0.288594,0.486111,...,-0.052871,0.173296,0.076763,0.536352,0.524493,0.011859,372.720173,234.122715,141.959375,0.016799


In [3]:
len(game_data)

630

In [4]:
# game data outputted from W1_DataCleaning.ipynb, no stats, goes back to 2005
game_data_no_stats = pd.read_csv('mydata/womens/matchups_no_stats.csv')
game_data_no_stats.head()

,Season,MooreRating_x,MooreRating_y,Seed_x,Seed_y,Upset,ScoreDiff,SeedDiff,MoorePredictedSpread
0,2005.0,86.08,66.75,5,12,0,22.0,-7,19.33
1,2005.0,86.08,84.71,5,4,0,9.0,1,1.37
2,2005.0,95.16,66.20,2,15,0,21.0,-13,28.96
3,2005.0,95.16,79.34,2,10,0,23.0,-8,15.82
4,2005.0,82.85,79.34,7,10,1,-3.0,-3,3.51


In [5]:
len(game_data_no_stats)

945

In [6]:
X_stats = game_data.drop(columns = ['Upset', 'ScoreDiff'])
y_stats = game_data['Upset']
X_no_stats = game_data_no_stats.drop(columns = ['Upset', 'ScoreDiff'])
y_no_stats = game_data_no_stats['Upset']

In [7]:
X_stats.describe()

,Season,MooreRating_x,MooreRating_y,Seed_x,3ptRate_x,Ast%_x,FT%_x,OppFT%_x,Opp3ptRate_x,OppAst%_x,...,xOffyOffFTRateDiff,AbsxOffyDefAstDiff,AbsyOffxDefAstDiff,xOffyDefAstAvg,yOffxDefAstAvg,xOffyOffAstDiff,TotalPossVarSum,GameScoreVarSum,AvgTotalPoss,MooreNaiveUpsetProbability
count,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,...,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,6.300000e+02
mean,2014.500000,91.447635,77.950905,3.501587,0.269958,0.580175,0.714094,0.685016,0.293521,0.525043,...,0.014510,0.074716,0.076593,0.558668,0.550156,0.008511,327.087600,237.323365,139.227468,1.954093e-01
std,2.874564,9.523172,10.045891,2.517822,0.068446,0.061336,0.041802,0.025582,0.043204,0.051228,...,0.044916,0.053422,0.053848,0.041075,0.039295,0.053630,86.016966,48.006064,4.608569,1.638898e-01
min,2010.000000,71.880000,50.100000,1.000000,0.083661,0.404508,0.549784,0.620482,0.170476,0.399135,...,-0.143509,0.000447,0.000066,0.458758,0.445061,-0.145324,168.263227,116.655992,124.592258,4.646013e-11
25%,2012.000000,84.525000,71.982500,1.000000,0.220174,0.537396,0.682909,0.667307,0.270069,0.486765,...,-0.014806,0.034642,0.033764,0.531133,0.522450,-0.029945,266.695298,202.993680,136.332350,3.303940e-02
50%,2014.500000,89.970000,78.455000,3.000000,0.271959,0.580132,0.713627,0.685484,0.291710,0.516972,...,0.011562,0.065343,0.067098,0.556035,0.548235,0.008747,312.734864,235.222923,139.124415,1.690637e-01
75%,2017.000000,96.700000,84.577500,5.000000,0.316067,0.623000,0.745287,0.700431,0.319613,0.562827,...,0.045697,0.105852,0.110597,0.588055,0.575420,0.045377,368.840148,268.182892,142.071003,3.384913e-01
max,2019.000000,118.220000,107.640000,12.000000,0.487848,0.766323,0.807420,0.745455,0.440853,0.705972,...,0.164782,0.339222,0.343439,0.687049,0.687824,0.189276,975.012208,377.864969,157.054323,4.992431e-01


In [8]:
X_no_stats.describe()

,Season,MooreRating_x,MooreRating_y,Seed_x,Seed_y,SeedDiff,MoorePredictedSpread
count,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000
mean,2012.000000,91.455471,78.216434,3.502646,9.089947,-5.587302,13.239037
std,4.322782,8.848588,9.930082,2.505977,4.323438,4.883500,11.306382
min,2005.000000,71.880000,50.100000,1.000000,1.000000,-15.000000,0.020000
25%,2008.000000,84.850000,72.230000,1.000000,6.000000,-9.000000,4.480000
50%,2012.000000,90.470000,78.610000,3.000000,9.000000,-5.000000,10.350000
75%,2016.000000,96.700000,84.660000,5.000000,13.000000,-1.000000,19.330000
max,2019.000000,118.220000,107.640000,12.000000,16.000000,8.000000,59.880000


In [9]:
# turns the probabilities for all games with a seed diff > 10 to one
def send_probs_to_one(preds, pred_feature_df, pred_indexes):
    seed_diffs = pred_feature_df['SeedDiff'][pred_indexes]
    for i in range(len(seed_diffs)):
        if seed_diffs[i] < -10:
            preds[i, 0] = 1
            preds[i, 1] = 0
        elif seed_diffs[i] > 10:
            preds[i, 0] = 0
            preds[i, 1] = 1
    
    return preds

### Logistic Regression

In [10]:
# lasso regularization for feature selection
# coefficients range from 0.02 to ~2000
# each coefficient is 6% larger than last, there are 125 coefficients in total
lasso_reg = [0.02 * (1.06) ** i for i in range(125)]

# ridge regularization to prevent overfitting
ridge_reg = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1]

# scale used to scale columns before applying regularization
scale = StandardScaler()

def train_logistic_regression(stats, features = None):
    
    # if using the stats dataframe or the non stats dataframe
    if stats:
        X = X_stats
        y = y_stats
        lasso_coef = lasso_reg
    else:
        X = X_no_stats
        y = y_no_stats
        lasso_coef = lasso_reg
        
    # list/dictionaries to store results
    feature_list = []
    scores = {}
    model_coefs = []
    models = []
    num_features = []
    
    # for each possible lasso regularization coefficient
    for c in tqdm(lasso_coef):
        
        if c is not None:
            # dont use season as a feature, use inputted features if given
            if features:
                X_lasso = X[features]
            else:
                X_lasso = X.drop(columns = ['Season'])

            # scale columns before regularization
            X_lasso = pd.DataFrame(scale.fit_transform(X_lasso), columns = X_lasso.columns)

            # fit L1 logistic regression for feature selection
            lasso = LogisticRegression(penalty = 'l1', C = c, random_state = 0, solver = 'saga', max_iter = 10000).fit(X_lasso, y)

            # filter for the columns with nonzero coefficients and the season column
            zero_cols = []
            if features:
                zero_cols = list(set(X.drop(columns = ['Season']).columns) - set(features))
            for i in range(len(lasso.coef_[0])):
                if lasso.coef_[0][i] == 0.0:
                    zero_cols.append(X_lasso.columns[i])
            nonzero_X = X.drop(columns = zero_cols)
        else:
            nonzero_X = X.copy()
        
        # if there is at least 1 nonzero column and the same amount of features hasn't already been built
        if (len(nonzero_X.columns) > 1 and (len(nonzero_X.columns) - 1) not in num_features):
            
            # try each L2 regularization coefficient
            for c2 in ridge_reg:
                
                # cross validate over each season
                for season in list(X['Season'].unique()):
                    
                    # add season to scores dictionary
                    if season not in scores:
                        scores[season] = []
                        
                    # split into train and validation sets
                    X_train = nonzero_X[nonzero_X['Season'] != season].drop(columns = ['Season'])
                    X_val = nonzero_X[nonzero_X['Season'] == season].drop(columns = ['Season'])
                    X_train = pd.DataFrame(scale.fit_transform(X_train), columns = X_train.columns)
                    X_val = pd.DataFrame(scale.transform(X_val), columns = X_val.columns)
                    y_train = y[X_train.index]
                    y_val = y[X_val.index]
                    
                    # fit logistic regression
                    log_model = LogisticRegression(penalty = 'l2', C = c2, max_iter = 10000, random_state = 0, solver = "sag").fit(X_train, y_train)
                    
                    # predict win probabilities
                    predictions = log_model.predict_proba(X_val)
                    
                    # send the favorites probabilities to one
                    predictions = send_probs_to_one(predictions, X, X_val.index)
                    
                    # calculate log loss and store
                    val_score = log_loss(y_val, predictions)
                    scores[season].append(val_score)
                    
                # retrain model on full dataset for coefficients
                log_model = LogisticRegression(penalty = 'l2', C = c2, max_iter = 10000, random_state = 0, solver = "sag").fit(pd.DataFrame(scale.fit_transform(nonzero_X), columns = nonzero_X.columns).drop(columns = ['Season']), y)
                
                # store model details
                feature_list.append(nonzero_X.drop(columns = ['Season']).columns)
                num_features.append(len(nonzero_X.drop(columns = ['Season']).columns))
                models.append(log_model)
                model_coefs.append({'lasso_coef': c, 'ridge_coef': c2})
                
    # if using stats dataframe, add nulls for scores of years we don't have data for
    if 2005 not in scores:
        scores[2005] = [None for i in range(len(models))]
        scores[2006] = [None for i in range(len(models))]
        scores[2007] = [None for i in range(len(models))]
        scores[2008] = [None for i in range(len(models))]
        scores[2009] = [None for i in range(len(models))]
                
    # return dataframe of results
    return pd.DataFrame({'Type': ['log' for i in range(len(models))],
                         'Num_Features': num_features,
                         'Features': feature_list,
                         'Model': models,
                         'Model_Coef': model_coefs,
                         'Stats': [stats for i in range(len(models))],
                         '2005_Score': scores[2005],
                         '2006_Score': scores[2006],
                         '2007_Score': scores[2007],
                         '2008_Score': scores[2008],
                         '2009_Score': scores[2009],
                         '2010_Score': scores[2010],
                         '2011_Score': scores[2011],
                         '2012_Score': scores[2012],
                         '2013_Score': scores[2013],
                         '2014_Score': scores[2014],
                         '2015_Score': scores[2015],
                         '2016_Score': scores[2016],
                         '2017_Score': scores[2017],
                         '2018_Score': scores[2018],
                         '2019_Score': scores[2019]})

### K-Nearest Neighbors

The purpose of the knn model is due to the high collinearity between SeedDiff and MooreRatingDiff being a problem for the logistic regression. Therefore, it will only use the dataframe without stats

In [11]:
ks = [10 + i for i in range(90)]
weights = ['uniform', 'distance']

def train_knn():
    
    # to store model info during cross validation
    scores = {}
    model_coefs = []
    models = []
    
    # for repeatable randomization
    random.seed(0)
    
    X = X_no_stats.copy()[['Season', 'SeedDiff', 'MoorePredictedSpread']]
    y = y_no_stats
    
    # train 50 models
    for k in tqdm(ks):
        for w in weights:
        
            knn_params = {'n_neighbors': k,
                         'weights': w}

            # for each season
            for season in list(X['Season'].unique()):

                # add season to scores dictionary
                if season not in scores:
                    scores[season] = []

                # split into train and validation sets
                X_train = X[X['Season'] != season].drop(columns = ['Season'])
                X_val = X[X['Season'] == season].drop(columns = ['Season'])
                X_train = pd.DataFrame(scale.fit_transform(X_train), columns = X_train.columns)
                X_val = pd.DataFrame(scale.transform(X_val), columns = X_val.columns)
                y_train = y[X_train.index]
                y_val = y[X_val.index]

                # fit knn
                knn_model = KNeighborsClassifier(n_neighbors = knn_params['n_neighbors'],
                                                 weights = knn_params['weights']).fit(X_train, y_train)

                # predict win probabilities
                predictions = knn_model.predict_proba(X_val)
                
                # send the favorites probabilities to one
                predictions = send_probs_to_one(predictions, X, X_val.index)

                # calculate log loss and store
                val_score = log_loss(y_val, predictions)
                scores[season].append(val_score)

            # store model details, no need to store KNN model yet
            models.append(None)
            model_coefs.append({'k': knn_params['n_neighbors'], 'weights': knn_params['weights']})
    
    # return dataframe of results
    return pd.DataFrame({'Type': ['knn' for i in range(len(models))],
                         'Num_Features': 'All',
                         'Features': 'All',
                         'Model': models,
                         'Model_Coef': model_coefs,
                         'Stats': [False for i in range(len(models))],
                         '2005_Score': scores[2005],
                         '2006_Score': scores[2006],
                         '2007_Score': scores[2007],
                         '2008_Score': scores[2008],
                         '2009_Score': scores[2009],
                         '2010_Score': scores[2010],
                         '2011_Score': scores[2011],
                         '2012_Score': scores[2012],
                         '2013_Score': scores[2013],
                         '2014_Score': scores[2014],
                         '2015_Score': scores[2015],
                         '2016_Score': scores[2016],
                         '2017_Score': scores[2017],
                         '2018_Score': scores[2018],
                         '2019_Score': scores[2019]})

### Results

In [12]:
# train the models
log_results1 = train_logistic_regression(True)
log_results2 = train_logistic_regression(False)
log_results3 = train_logistic_regression(False, ['SeedDiff', 'MoorePredictedSpread'])
log_results4 = train_logistic_regression(True, ['SeedDiff', 'MoorePredictedSpread', 'MooreNaiveUpsetProbability', '3pt%Var_y'])
knn_results = train_knn()
final_results = pd.concat([log_results1, log_results2, log_results3, log_results4, knn_results], ignore_index = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:44<00:00,  2.01it/s]


In [28]:
# calculate percentiles of scores in relation to other models
score_cols = []
for col in final_results.columns:
    if col.endswith("Score") and not col.startswith('A') and int(col.split('_')[0]) > 2009:
        score_cols.append(col)

In [29]:
# calculate average score and average percentile
final_results['Avg_Score'] = final_results[score_cols].mean(axis = 1)

In [30]:
# function to show params of given model by results row index
def params(row_index):
    return final_results.loc[row_index, 'Model_Coef']

In [31]:
# function to show feature importances of given model by results row index
def features(row_index):
    
    # if model is logistic regression, return coefficients
    if final_results.loc[row_index, 'Type'] == 'log':
        
        model = final_results.loc[row_index, 'Model']
        model_features = list(final_results.loc[row_index, 'Features'])
        sign_list = []
        coef_list = []
        for i in range(len(model_features)):
            coef = round(model.coef_[0][i], 6)
            coef_list.append(coef)
            if coef > 0:
                sign_list.append('+')
            else:
                sign_list.append('-')
        
        return_df =  pd.DataFrame({'Feature': model_features,
                             'Sign': sign_list,
                            'Coefficient': coef_list})
        return return_df.reindex(return_df['Coefficient'].abs().sort_values(ascending = False).index)
        
    # KNN doesn't really have a feature importance
    else:
        print('All features have equal weight in KNN')

In [32]:
# print results
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
final_results.sort_values(by = ['Avg_Score'], ascending = True)

,Type,Num_Features,Features,Model,Model_Coef,Stats,2005_Score,2006_Score,2007_Score,2008_Score,2009_Score,2010_Score,2011_Score,2012_Score,2013_Score,2014_Score,2015_Score,2016_Score,2017_Score,2018_Score,2019_Score,Avg_Score
639,log,2,"Index(['MooreRating_x', 'MoorePredictedSpread'...","LogisticRegression(C=1e-07, max_iter=10000, ra...","{'lasso_coef': 0.042658565202913695, 'ridge_co...",False,0.449434,0.442765,0.442512,0.440136,0.44156,0.437614,0.439959,0.437820,0.437386,0.437819,0.437820,0.437916,0.437473,0.437914,0.445678,0.438740
651,log,3,"Index(['MooreRating_x', 'SeedDiff', 'MoorePred...","LogisticRegression(C=1e-07, max_iter=10000, ra...","{'lasso_coef': 0.08583741439486976, 'ridge_coe...",False,0.449434,0.447999,0.44603,0.440134,0.441559,0.441821,0.441558,0.441559,0.438122,0.438326,0.437820,0.437916,0.437473,0.437915,0.437420,0.438993
663,log,4,"Index(['MooreRating_x', 'Seed_x', 'SeedDiff', ...","LogisticRegression(C=1e-07, max_iter=10000, ra...","{'lasso_coef': 1.7765671239028533, 'ridge_coef...",False,0.450596,0.447999,0.44603,0.445845,0.444989,0.441821,0.441908,0.443013,0.438122,0.438325,0.437820,0.437916,0.437474,0.438697,0.438776,0.439387
687,log,2,"Index(['SeedDiff', 'MoorePredictedSpread'], dt...","LogisticRegression(C=1e-07, max_iter=10000, ra...","{'lasso_coef': 0.08097869282534884, 'ridge_coe...",False,0.4475,0.446034,0.439524,0.440137,0.440138,0.440581,0.437825,0.502976,0.438123,0.438327,0.443154,0.444079,0.437471,0.445674,0.438235,0.446645
676,log,1,"Index(['MoorePredictedSpread'], dtype='object')","LogisticRegression(C=1e-06, max_iter=10000, ra...","{'lasso_coef': 0.02, 'ridge_coef': 1e-06}",False,0.451828,0.451297,0.451176,0.448783,0.448786,0.449808,0.451175,0.442820,0.449511,0.448352,0.451047,0.451047,0.448783,0.447340,0.445380,0.448526
628,log,1,"Index(['MoorePredictedSpread'], dtype='object')","LogisticRegression(C=1e-06, max_iter=10000, ra...","{'lasso_coef': 0.02, 'ridge_coef': 1e-06}",False,0.451828,0.451297,0.451176,0.448783,0.448786,0.449808,0.451175,0.442820,0.449511,0.448352,0.451047,0.451047,0.448783,0.447340,0.445380,0.448526
688,log,2,"Index(['SeedDiff', 'MoorePredictedSpread'], dt...","LogisticRegression(C=1e-06, max_iter=10000, ra...","{'lasso_coef': 0.08097869282534884, 'ridge_coe...",False,0.452093,0.451651,0.45117,0.448773,0.451655,0.451168,0.451167,0.449812,0.451176,0.448341,0.451301,0.451040,0.451036,0.450885,0.450717,0.450664
640,log,2,"Index(['MooreRating_x', 'MoorePredictedSpread'...","LogisticRegression(C=1e-06, max_iter=10000, ra...","{'lasso_coef': 0.042658565202913695, 'ridge_co...",False,0.452163,0.451288,0.451165,0.451167,0.451649,0.451163,0.451165,0.451294,0.451174,0.449485,0.451035,0.451044,0.451044,0.451028,0.450706,0.450914
652,log,3,"Index(['MooreRating_x', 'SeedDiff', 'MoorePred...","LogisticRegression(C=1e-06, max_iter=10000, ra...","{'lasso_coef': 0.08583741439486976, 'ridge_coe...",False,0.452091,0.452055,0.451159,0.451162,0.451649,0.451157,0.451157,0.451399,0.451170,0.449476,0.451291,0.451036,0.451036,0.451022,0.450699,0.450944
664,log,4,"Index(['MooreRating_x', 'Seed_x', 'SeedDiff', ...","LogisticRegression(C=1e-06, max_iter=10000, ra...","{'lasso_coef': 1.7765671239028533, 'ridge_coef...",False,0.452162,0.452055,0.451156,0.451864,0.451648,0.451156,0.451632,0.451493,0.451172,0.449473,0.451395,0.451036,0.451044,0.451020,0.450695,0.451012


In [18]:
features(639)

,Feature,Sign,Coefficient
1,MoorePredictedSpread,-,-0.000013
0,MooreRating_x,-,-0.000008


In [19]:
features(651)

,Feature,Sign,Coefficient
2,MoorePredictedSpread,-,-0.000013
1,SeedDiff,+,0.000008
0,MooreRating_x,-,-0.000008


In [20]:
features(663)

,Feature,Sign,Coefficient
3,MoorePredictedSpread,-,-0.000013
2,SeedDiff,+,0.000008
0,MooreRating_x,-,-0.000008
1,Seed_x,+,0.000007


In [25]:
final_results.loc[651, 'Features']

Index(['MooreRating_x', 'SeedDiff', 'MoorePredictedSpread'], dtype='object')

In [26]:
pickle.dump(final_results.loc[651, 'Model'], open('models/WProb.sav', 'wb'))